In [1]:
import nltk

nltk.download('reuters')

[nltk_data] Downloading package reuters to /root/nltk_data...


True

In [2]:
from nltk.corpus import reuters
print('training files : ', len([fid for fid in reuters.fileids() if fid[:5] == 'train']))
print('testing files : ', len([fid for fid in reuters.fileids() if fid[:4] == 'test']))

training files :  7769
testing files :  3019


In [3]:
# Fonction pour obtenir le texte du document en supprimant les sauts de ligne '\n'
def get_text(fileid):
    return reuters.raw(fileid).replace('\n', ' ')

# Fonction qui retourne le label 1 pour 'grain' et 0 sinon
def get_label(fileid):
    return 1 if 'grain' in reuters.categories(fileid) else 0

In [4]:
import pandas as pd
from nltk.corpus import reuters

# Récupération des différents documents en fonction du nom de fichier
train_data = [[get_text(fileid), get_label(fileid)] for fileid in reuters.fileids() if fileid.startswith('train')]
test_data = [[get_text(fileid), get_label(fileid)] for fileid in reuters.fileids() if fileid.startswith('test')]

# Création des DataFrames pandas
train = pd.DataFrame(train_data, columns=['text', 'label'])
test = pd.DataFrame(test_data, columns=['text', 'label'])

In [5]:
# Sélection de la pipeline à tester
def set_search(experimental=False):
    pipeline = experimental_pipeline if experimental else default_pipeline
    parameter_grid = (parameter_grid_experimental if experimental
                      else parameter_grid_default)

    search = GridSearchCV(
        pipeline,
        parameter_grid,
        scoring='f1_micro',
        cv=3,
        n_jobs=-1,
        verbose=2
    )
    print("Testing pipeline:", pipeline)
    return search, parameter_grid

# Entraînement de la pipeline puis calcul des performances sur les données de tesst
def test_data(search):
    print("Evaluating pipeline...")
    start = time()
    search.fit(train['text'], train['label'])
    end = time()
    print(f"Training time: {end - start:.3f}s")

    test_accuracy = search.score(test['text'], test['label'])

    print(f"Accuracy on test set: {test_accuracy:.3f}")

# Affichage des meilleurs hyperparamètres trouvés
def display_params(search, parameter_grid):
    print("Best parameters combination found:")
    best_parameters = search.best_estimator_.get_params()
    for param_name in sorted(parameter_grid.keys()):
        print(f"{param_name}: {best_parameters[param_name]}")

# Création de la pipeline, entraînement, test et affichage des paramètres trouvés
def search_best_parameters(experimental=False):
    [search, parameter_grid] = set_search(experimental)

    test_data(search)
    display_params(search, parameter_grid)

In [ ]:
from time import time
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

default_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", ComplementNB()),
    ]
)

experimental_pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("clf", SGDClassifier(loss='hinge'))
    ]
)

parameter_grid_default = {
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__ngram_range": ((1, 1), (1, 2)),
    "vect__norm": ("l1", "l2"),  # normalization options for TFIDF
    "clf__alpha": np.logspace(-6, 6, 13),
}

parameter_grid_experimental = {
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__ngram_range": ((1, 1), (1, 2)),
    "clf__alpha": np.logspace(-6, 6, 13),
}

for experimental in [False, True]:
    search_best_parameters(experimental)

Testing pipeline: Pipeline(steps=[('vect', TfidfVectorizer()), ('clf', ComplementNB())])
Evaluating pipeline...
Fitting 3 folds for each of 1040 candidates, totalling 3120 fits


In [ ]:
#TODO :
# 1) Présenter le score f1 sur l'ensemble de test avec la meilleure configuration trouvée
# 2) Présenter la loss et l'accuracy
# 3) Pourquoi pas ajouter une matrice de confusion sur les résultats de la classe "grain" (l'assistant aimerait bien à mon avis)